![An interactive LADAL notebook](https://slcladal.github.io/images/uq1.jpg)


This tutorial is the interactive Jupyter notebook accompanying the [*Language Technology and Data Analysis Laboratory* (LADAL) tutorial *Introduction to Data Visualization in R*](https://ladal.edu.au/introviz.html). 


**Preparation and session set up**

We set up our session by activating the packages we need for this tutorial. 


In [ ]:
library(stringr)
library(dplyr)
library(tidyr)
library(ggplot2)
library(gridExtra)
library(RColorBrewer)
library(openxlsx)


Once you have initiated the session by executing the code shown above, you are good to go.

If you are using this notebook on your own computer and you have not already installed the R packages listed above, you need to install them. You can install them by replacing the `library` command with `install.packages` and putting the name of the package into quotation marks like this: `install.packages("stringr")`. Then, you simply run this command and R will install the package you specified.

# Getting started

Before turning to the graphs, we load the data (see the [LADAL tutorial](https://ladal.edu.au/introviz.html) for more information about what the data represents). 


In [ ]:
# load data
pdat  <- base::readRDS(url("https://slcladal.github.io/data/pvd.rda", "rb"))
# inspect data
head(pdat)


***

## Using your own data

While the tutorial uses data from the LADAL website, you can also use your own data. You can see below what you need to do to upload and use your own data.

The code chunk below allows you to upload two files from your own computer. To be able to load your own data, you need to click on the folder symbol to the left of the screen:

![Binder Folder Symbol](https://slcladal.github.io/images/binderfolder.JPG)


Then on the upload symbol.

![Binder Upload Symbol](https://slcladal.github.io/images/binderupload.JPG)

Next, upload the files you want to analyze and then the respective files names in the file argument of the scan function. When you then execute the code (like to code chunk below, you will upload your own data.


In [ ]:
mytable1 <- openxlsx::read.xlsx("testdata1.xlsx", sheet = 1)
# inspect
mytable1


**Keep in mind though that you need to adapt the names of the texts in the code chunks below so that the code below work on your own texts!**

***



We will now turn to creating the graphs.

# Creating a simple graph

When creating a visualization with ggplot, we first use the function `ggplot` and define the data that the visualization will use, then, we define the aesthetics which define the layout, i.e. the x- and y-axes. 


In [ ]:
ggplot(pdat, aes(x = Date, y = Prepositions))



In a next step, we add the geom-layer which defines the type of visualization that we want to display. In this case, we use `geom_point` as we want to show points that stand for the frequencies of prepositions in each text. Note that we add the geom-layer by adding a `+` at the end of the line!



In [ ]:
ggplot(pdat, aes(x = Date, y = Prepositions)) +
  geom_point()


We can also add another layer, e.g. a layer which shows a smoothed loess line, and we can change the theme by specifying the theme we want to use. Here, we will use `theme_bw` which stands for the black-and-white theme (we will get into the different types of themes later).



In [ ]:
ggplot(pdat, aes(x = Date, y = Prepositions)) +
  geom_point() +
  geom_smooth(se = F) +
  theme_bw()


We can also store our plot in an object and then add different layers to it or modify the plot. Here we store the basic graph in an object that we call `p` and then change the axes names.  



In [ ]:
# store plot in object p
p <- ggplot(pdat, aes(x = Date, y = Prepositions)) +
  geom_point() +
  theme_bw()
# add layer with nicer axes titles to p
p + labs(x = "Year", y = "Frequency")


We can also integrate plots into data processing pipelines as shown below. When you integrate visualizations into pipelines, you should not specify the data as it is clear from the pipe which data the plot is using.



In [ ]:
pdat %>%
  dplyr::select(DateRedux, GenreRedux, Prepositions) %>%
  dplyr::group_by(DateRedux, GenreRedux) %>%
  dplyr::summarise(Frequency = mean(Prepositions)) %>%
    ggplot(aes(x = DateRedux, y = Frequency, group = GenreRedux, color = GenreRedux)) +
    geom_line()


# Modifying axes and titles

There are different way to modify axes, the easiest way is to specify the axes labels using `labs` (as already shown above). To add a custom title, we can use `ggtitle`.


In [ ]:
p + labs(x = "Year", y = "Frequency") +
  ggtitle("Preposition use over time", subtitle="based on the PPC corpus")


To change the range of the axes, we can specify their limits in the `coord_cartesian` layer.



In [ ]:
p + coord_cartesian(xlim = c(1000, 2000), ylim = c(-100, 300))



In [ ]:
p + 
  labs(x = "Year", y = "Frequency") +
  theme(axis.text.x = element_text(face="italic", color="red", size=8, angle=45),
          axis.text.y = element_text(face="bold", color="blue", size=15, angle=90))


In [ ]:
p + theme(
  axis.text.x = element_blank(),
  axis.text.y = element_blank(),
  axis.ticks = element_blank())


In [ ]:
p + scale_x_discrete(name ="Year of composition", limits=seq(1150, 1900, 50)) +
  scale_y_discrete(name ="Relative Frequency", limits=seq(70, 190, 20))


# Modifying colors

To modify colors, you can include a color specification in the main aesthetics.


In [ ]:
ggplot(pdat, aes(x = Date, y = Prepositions, color = GenreRedux)) +
  geom_point() 


Or you can specify the color in the aesthetics of the geom-layer.



In [ ]:
p + geom_point(aes(color = GenreRedux))



To change the default colors manually, you can use `scale_color_manual` and define the colors you want to use in the `values` argument and specify the variable levels that want to distinguish by colors in the `breaks` argument. You can find an overview of the colors that you can define in R [here](http://www.stat.columbia.edu/~tzheng/files/Rcolor.pdf).



In [ ]:
ggplot(pdat, aes(x = Date, y = Prepositions, color = GenreRedux)) +
  geom_point()  + 
  scale_color_manual(values = c("red", "gray30", "blue", "orange", "gray80"),
                       breaks = c("Conversational", "Fiction", "Legal", "NonFiction", "Religious"))


When the variable that you want to colorize does not have discrete levels, you use `scale_color_continuous` instead of `scale_color_manual`.



In [ ]:
p + geom_point(aes(color = Prepositions)) + 
  scale_color_continuous()


You can also change colors by specifying color `palettes`. Color `palettes` are predefined vectors of colors and there are many different color `palettes` available. Below are some examples using the `Brewer` color palette.



In [ ]:
p + geom_point(aes(color = GenreRedux)) + 
  scale_color_brewer()


In [ ]:
p + geom_point(aes(color = GenreRedux)) + 
  scale_color_brewer(palette = 2)


In [ ]:
p + geom_point(aes(color = GenreRedux)) + 
  scale_color_brewer(palette = 3)


We now use the `viridis` color palette to show how you can use another palette. The example below uses the viridis palette for a discrete variable (GenreRedux).



In [ ]:
p + geom_point(aes(color = GenreRedux)) + 
  scale_color_viridis_d()


To use the viridis palette for continuous variables you need to use `scale_color_viridis_c` instead of `scale_color_viridis_d`.



In [ ]:
p + geom_point(aes(color = Prepositions)) + 
  scale_color_viridis_c()


The `Brewer` color palette (see below) is the most commonly used color palette but there are many more. You can find an overview of the color palettes that are available [here](https://www.datanovia.com/en/blog/top-r-color-palettes-to-know-for-great-data-visualization/).



In [ ]:
display.brewer.all()



# Modifying lines & symbols



In [ ]:
ggplot(pdat, aes(x = Date, y = Prepositions, shape = GenreRedux)) +
  geom_point() 


In [ ]:
ggplot(pdat, aes(x = Date, y = Prepositions)) + 
  geom_point(aes(shape = GenreRedux)) + 
  scale_shape_manual(values = 1:5)


Similarly, if you want to change the lines in a line plot, you define the `linetype` in the aesthetics.



In [ ]:
pdat %>%
  dplyr::select(GenreRedux, DateRedux, Prepositions) %>%
  dplyr::group_by(GenreRedux, DateRedux) %>%
  dplyr::summarize(Frequency = mean(Prepositions)) %>%
  ggplot(aes(x = DateRedux, y = Frequency, group = GenreRedux, linetype = GenreRedux)) +
  geom_line()


You can of course also manually specify the line types.



In [ ]:
pdat %>%
  dplyr::select(GenreRedux, DateRedux, Prepositions) %>%
  dplyr::group_by(GenreRedux, DateRedux) %>%
  dplyr::summarize(Frequency = mean(Prepositions)) %>%
  ggplot(aes(x = DateRedux, y = Frequency, group = GenreRedux, linetype = GenreRedux)) +
  geom_line() +
  scale_linetype_manual(values = c("twodash", "longdash", "solid", "dotted", "dashed"))


Here is an overview of the most commonly used linetypes in R.



In [ ]:
d=data.frame(lt=c("blank", "solid", "dashed", "dotted", "dotdash", "longdash", "twodash", "1F", "F1", "4C88C488", "12345678"))
ggplot() +
scale_x_continuous(name="", limits=c(0,1)) +
scale_y_discrete(name="linetype") +
scale_linetype_identity() +
geom_segment(data=d, mapping=aes(x=0, xend=1, y=lt, yend=lt, linetype=lt))


To make your layers transparent, you need to specify `alpha` values.



In [ ]:
ggplot(pdat, aes(x = Date, y = Prepositions)) + 
  geom_point(alpha = .2)


Transparency can be particularly useful when using different layers that add different types of visualizations.



In [ ]:
ggplot(pdat, aes(x = Date, y = Prepositions)) + 
  geom_point(alpha = .1) + 
  geom_smooth(se = F)


Transparency can also be linked to other variables.



In [ ]:
ggplot(pdat, aes(x = Date, y = Prepositions, alpha = Region)) + 
  geom_point()


In [ ]:
ggplot(pdat, aes(x = Date, y = Prepositions, alpha = Prepositions)) + 
  geom_point()


# Adapting sizes



In [ ]:
ggplot(pdat, aes(x = Date, y = Prepositions, size = Region, color = GenreRedux)) +
  geom_point() 


In [ ]:
ggplot(pdat, aes(x = Date, y = Prepositions, color = GenreRedux, size = Prepositions)) +
  geom_point() 


# Adding text



In [ ]:
pdat %>%
  dplyr::filter(Genre == "Fiction") %>%
  ggplot(aes(x = Date, y = Prepositions, label = Prepositions, color = Region)) +
  geom_text(size = 3) +
  theme_bw()


In [ ]:
pdat %>%
  dplyr::filter(Genre == "Fiction") %>%
  ggplot(aes(x = Date, y = Prepositions, label = Prepositions)) +
  geom_text(size = 3, hjust=1.2) +
  geom_point() +
  theme_bw()


In [ ]:
pdat %>%
  dplyr::filter(Genre == "Fiction") %>%
  ggplot(aes(x = Date, y = Prepositions, label = Prepositions)) +
  geom_text(size = 3, nudge_x = -15, check_overlap = T) +
  geom_point() +
  theme_bw()


In [ ]:
pdat %>%
  dplyr::filter(Genre == "Fiction") %>%
  ggplot(aes(x = Date, y = Prepositions, label = Prepositions)) +
  geom_text(size = 3, nudge_x = -15, check_overlap = T) +
  geom_point() +
  theme_bw()


In [ ]:
ggplot(pdat, aes(x = Date, y = Prepositions)) +
  geom_point() +
  ggplot2::annotate(geom = "text", label = "Some text", x = 1200, y = 175, color = "orange") +
  ggplot2::annotate(geom = "text", label = "More text", x = 1850, y = 75, color = "lightblue", size = 8) +
    theme_bw()


In [ ]:
pdat %>%
  dplyr::group_by(GenreRedux) %>%
  dplyr::summarise(Frequency = round(mean(Prepositions), 1)) %>%
  ggplot(aes(x = GenreRedux, y = Frequency, label = Frequency)) +
  geom_bar(stat="identity") +
  geom_text(vjust=-1.6, color = "black") +
  coord_cartesian(ylim = c(0, 180)) +
  theme_bw()


In [ ]:
pdat %>%
  dplyr::group_by(Region, GenreRedux) %>%
  dplyr::summarise(Frequency = round(mean(Prepositions), 1)) %>%
  ggplot(aes(x = GenreRedux, y = Frequency, group = Region, fill = Region, label = Frequency)) +
  geom_bar(stat="identity", position = "dodge") +
  geom_text(vjust=1.6, position = position_dodge(0.9)) + 
  theme_bw()


In [ ]:
pdat %>%
  dplyr::filter(Genre == "Fiction") %>%
  ggplot(aes(x = Date, y = Prepositions, label = Prepositions)) +
  geom_label(size = 3, vjust=1.2) +
  geom_point() +
  theme_bw()


# Combining multiple plots



In [ ]:
ggplot(pdat, aes(x = Date, y = Prepositions)) +
  facet_grid(~GenreRedux) +
  geom_point() + 
  theme_bw()


In [ ]:
ggplot(pdat, aes(x = Date, y = Prepositions)) +
  facet_wrap(vars(Region, GenreRedux), ncol = 5) +
  geom_point() + 
  theme_bw()


In [ ]:
p1 <- ggplot(pdat, aes(x = Date, y = Prepositions)) + geom_point() + theme_bw()
p2 <- ggplot(pdat, aes(x = GenreRedux, y = Prepositions)) + geom_boxplot() + theme_bw()
p3 <- ggplot(pdat, aes(x = DateRedux, group = GenreRedux)) + geom_bar() + theme_bw()
p4 <- ggplot(pdat, aes(x = Date, y = Prepositions)) + geom_point() + geom_smooth(se = F) + theme_bw()
grid.arrange(p1, p2, nrow = 1)


In [ ]:
grid.arrange(grobs = list(p4, p2, p3), 
             widths = c(2, 1), 
             layout_matrix = rbind(c(1, 1), c(2, 3)))


# Available themes



In [ ]:
p <- ggplot(pdat, aes(x = Date, y = Prepositions)) + geom_point() + labs(x = "", y= "") +
  ggtitle("Default") + theme(axis.text.x = element_text(size=6, angle=90))
p1 <- p + theme_bw() + ggtitle("theme_bw") + theme(axis.text.x = element_text(size=6, angle=90))
p2 <- p + theme_classic() + ggtitle("theme_classic") + theme(axis.text.x = element_text(size=6, angle=90))
p3 <- p + theme_minimal() + ggtitle("theme_minimal") + theme(axis.text.x = element_text(size=6, angle=90))
p4 <- p + theme_light() + ggtitle("theme_light") + theme(axis.text.x = element_text(size=6, angle=90))
p5 <- p + theme_dark() + ggtitle("theme_dark") + theme(axis.text.x = element_text(size=6, angle=90))
p6 <- p + theme_void() + ggtitle("theme_void") + theme(axis.text.x = element_text(size=6, angle=90))
p7 <- p + theme_gray() + ggtitle("theme_gray") + theme(axis.text.x = element_text(size=6, angle=90))
grid.arrange(p, p1, p2, p3, p4, p5, p6, p7, ncol = 4)


In [ ]:
ggplot(pdat, aes(x = Date, y = Prepositions, color = GenreRedux)) +
  geom_point() + 
  theme(panel.background = element_rect(fill = "white", colour = "red"))


Extensive information about how to modify themes can be found  [here](https://ggplot2.tidyverse.org/reference/theme.html).

# Modifying legends


In [ ]:
ggplot(pdat, aes(x = Date, y = Prepositions, color = GenreRedux)) +
  geom_point() + 
  theme(legend.position = "top")


In [ ]:
ggplot(pdat, aes(x = Date, y = Prepositions, color = GenreRedux)) +
  geom_point() + 
  theme(legend.position = "none")


In [ ]:
ggplot(pdat, aes(x = Date, y = Prepositions, linetype = GenreRedux, color = GenreRedux)) +
  geom_smooth(se = F) +  
  theme(legend.position = c(0.2, 0.7)) 


In [ ]:
ggplot(pdat, aes(x = Date, y = Prepositions, linetype = GenreRedux, color = GenreRedux)) +
  geom_smooth(se = F) + 
  guides(color=guide_legend(override.aes=list(fill=NA))) +  
  theme(legend.position = "top", 
        legend.text = element_text(color = "green")) +
  scale_linetype_manual(values=1:5, 
                        name=c("Genre"),
                        breaks = names(table(pdat$GenreRedux)),
                        labels = names(table(pdat$GenreRedux))) + 
  scale_colour_manual(values=c("red", "gray30", "blue", "orange", "gray80"),
                      name=c("Genre"),
                      breaks=names(table(pdat$GenreRedux)),  
                      labels = names(table(pdat$GenreRedux)))


***

[Back to LADAL](https://ladal.edu.au/introviz.html)

***
